In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

In [5]:
tree_grid1 = {"n_estimators" : [20, 40, 60, 80, 100],
            "learning_rate" : [0.01, 0.05, 0.1]}
tree_grid2 = {"n_estimators" : [20, 40, 60, 80, 100],
             "max_features" : ["log2", "sqrt"]}
linear_grid1 = {"penalty" : ["l2", "l1"], 
                "C" : [0.6, 0.7, 0.8, 0.9, 1.0]}
linear_grid2 = {"loss" : ["modified_huber", "squared_hinge", "perceptron"],
               "penalty" : ["l2", "l1"]}

models = [CatBoostClassifier(verbose = False), LGBMClassifier(), RandomForestClassifier(), LogisticRegression(solver = "liblinear"), SGDClassifier()]
model_name = ["CAT ", "LGBM ", "RF ", "LR ", "SGD "]
pdict = {0 : tree_grid1, 1 : tree_grid1, 2 : tree_grid2, 3 : linear_grid1, 4 : linear_grid2}

for l1 in ["o"]:
    for l2 in ["i", "b", "f"]:
        for n in range(1, 5):
            DATA_PATH = l1 + l2 + "_train" + str(n) + ".csv"
            df = pd.read_csv(DATA_PATH)
            X, y = df.drop(["target"], axis = 1), df["target"]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 24)
            idx = 0
            for model in models:
                grid = GridSearchCV(model, param_grid = pdict[idx], cv = 3, refit = True)
                grid.fit(X_train, y_train)
                _pred = grid.best_estimator_.predict(X_test)
                base = accuracy_score(y_test, _pred)
                res = []
                for col in X_train.columns:
                    temp_res = []
                    for _ in range(10):
                        save = X_test[col].copy()
                        X_test[col] = np.random.permutation(X_test[col])
                        temp_res.append(base - accuracy_score(y_test, grid.best_estimator_.predict(X_test)))
                    X_test[col] = save
                    res.append([col, np.mean(temp_res)])
                res = sorted(res, key = lambda x : x[1], reverse = True)
                col_lst = []
                for data in res:
                    if data[1] <= 0: break
                    else: col_lst.append(data[0])
                if len(col_lst) == 0: col_lst = X_train.columns
                X_train = X_train.loc[:, col_lst]; X_test = X_test.loc[:, col_lst]
                X = pd.concat([X_train, X_test], axis = 0)
                y = pd.concat([y_train, y_test], axis = 0); y.columns = ["target"]
                data = pd.concat([X, y], axis = 1)
                data.to_csv(model_name[idx] + DATA_PATH[:-4] + "-0-filtered.csv", index = False)
                idx += 1

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
